In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from analysis.processor import Processor

In [3]:
from utils.filesysutil import *

In [14]:
from analysis.custom import *

In [5]:
from config.selectionconfig import runsetting as rs

In [6]:
import random
import json
import os

In [7]:
import awkward as ak

## Inspect a skimmed file and run custom selections

In [50]:
filename = 'GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_1-part0.root'
dirname = '/uscms/home/joyzhou/nobackup/output/ggF'
proc = Processor(rs, 'ggF', prelimEvtSel)

In [51]:
evts = proc.loadfile(pjoin(dirname, filename), 0)
# Do something after this cell to conduct inspection

In [52]:
evts['Tau_pt'].compute()

<Array [[115, 20.6, 24.4], ..., [123, 22.9]] type='33954 * var * float32'>

In [53]:
proc.runfile(pjoin(dirname, filename), 0)

'file /uscms/home/joyzhou/nobackup/output/ggF/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_1-part0.root processed successfully!'

In [54]:
result = pjoin('/uscms/home/joyzhou/nobackup/tests', 'ggF_0-part0.root')
fit_evts = proc.loadfile(result, 'test')

## Run over example files

In [9]:
# custom-define datasets to process 
process_names = ["TTbar", "ggF", "SingleH"]
data_src = 'src/data/preprocessed'

In [9]:
# Quick hack
ds_names = ["DYJets", "TTbar", "ggF"]
filelist = ["root://cmsxrootd.fnal.gov//store/mc/Run3Winter22NanoAOD/DYJetsToLL_M-10to50_TuneCP5_13p6TeV-madgraphMLM-pythia8/NANOAODSIM/122X_mcRun3_2021_realistic_v9-v1/30000/10e16df1-d8bb-4b32-bb90-32c4fd57f429.root",
           "root://cmsxrootd.fnal.gov//store/mc/Run3Summer22EENanoAODv12/TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v2/2540000/62aea484-cae5-4c41-8803-08969f44c116.root",
           "root://cmsxrootd.fnal.gov//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v3/80000/5998ad11-ab82-41bf-a740-7a2c38b85a14.root"]

In [10]:
finame = filelist[1]
proc = Processor(rs, 'TTbar')

In [18]:
events = proc.loadfile(finame, 'test')

In [36]:
proc.outdir

'/uscms/home/joyzhou/nobackup/tests'

In [37]:
proc = Processor(rs, 'TTbar', mockskimEvtSel)

In [ ]:
# examine the file later
proc.runfile()

In [17]:
# create an object mask
tau_mask = (events['Tau_pt'] > 25) & (abs(events['Tau_eta']) < 3.0)

In [9]:
for i, ds in enumerate(ds_names):
    proc = Processor(rs, ds)
    proc.runfile(filelist[i], 0, None)

In [15]:
output_fi = os.listdir(rs.OUTPUTDIR_PATH)

In [21]:
dfs = []
for file_name in output_fi:
    df = pd.read_csv(pjoin(rs.OUTPUTDIR_PATH, file_name), index_col=0, header=0)
    df.columns = [file_name]
    dfs.append(df)

In [22]:
raw_df = pd.concat(dfs, axis=1)

In [23]:
raw_df

,DYJets_cutflow_0.csv,TTbar_cutflow_0.csv,ggF_cutflow_0.csv
initial,506334,927960,80104
ElectronSelection,500522,525938,45205
MuonSelection,488033,289438,31124
TauSelection,361,32286,14806


In [24]:
efficiency_df = raw_df.div(raw_df.shift(1)).fillna(1)

In [25]:
# incremental
efficiency_df

,DYJets_cutflow_0.csv,TTbar_cutflow_0.csv,ggF_cutflow_0.csv
initial,1.000000,1.000000,1.000000
ElectronSelection,0.988521,0.566768,0.564329
MuonSelection,0.975048,0.550327,0.688508
TauSelection,0.000740,0.111547,0.475710


In [26]:
overall_eff = raw_df.div(raw_df.iloc[0]).fillna(1)

In [27]:
# wrt to total
overall_eff

,DYJets_cutflow_0.csv,TTbar_cutflow_0.csv,ggF_cutflow_0.csv
initial,1.000000,1.000000,1.000000
ElectronSelection,0.988521,0.566768,0.564329
MuonSelection,0.963856,0.311908,0.388545
TauSelection,0.000713,0.034792,0.184835


In [ ]:
# Run small scale selections on one file for each process
def runsample(dsnames, datasrc):
    samplefl = []
    for name in dsnames:
        with open(pjoin(datasrc, f'{name}.json'), 'r') as file:
            meta = json.load(file)
            samplefl.append(meta)